In [1]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import pandas as pd

books=pd.read_csv("cleaned_books.csv")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_with_subtile,tagged_isbn13_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006163831,0006163831,The One Tree,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,The One Tree,9780006163831 Volume Two of Stephen Donaldson'...
3,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
4,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
6350,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
6351,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...
6352,9789027712059,9027712050,The Berlin Phenomenology,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0,The Berlin Phenomenology,9789027712059 Since the three volume edition o...


In [4]:
import  csv
books["tagged_isbn13_description"].to_csv(
    "tagged_isbn13_description.txt",
    index=False,
    header=False,
    sep="\n",
    quoting=csv.QUOTE_NONE,
    escapechar="\\"
)

In [5]:

raw_documents = TextLoader("tagged_isbn13_description.txt").load()

splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n",length_function=len)

documents=splitter.split_documents(raw_documents)


Created a chunk of size 1168, which is longer than the specified 1
Created a chunk of size 1214, which is longer than the specified 1
Created a chunk of size 123, which is longer than the specified 1
Created a chunk of size 373, which is longer than the specified 1
Created a chunk of size 309, which is longer than the specified 1
Created a chunk of size 479, which is longer than the specified 1
Created a chunk of size 482, which is longer than the specified 1
Created a chunk of size 960, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 128, which is longer than the specified 1
Created a chunk of size 843, which is longer than the specified 1
Created a chunk of size 284, which is longer than the specified 1
Created a chunk of size 193, which is longer than the specified 1
Created a chunk of size 877, which is longer than the specified 1
Created a chunk of size 1088, which is longer than the specified 1
Created

In [6]:
for i, doc in enumerate(documents):
    if doc.page_content[0]=="\"":
        print(f"Doc {i}: {doc.page_content[:100]}")

In [7]:
len(documents)

6354

In [8]:
db_books=Chroma.from_documents(documents, embedding=MistralAIEmbeddings())

In [9]:
query = "A book to teach children about nature"
res = db_books.similarity_search(query, k=10)
res

[Document(id='1d0d429b-f7c2-4dd6-ae02-6a6714db8df5', metadata={'source': 'tagged_isbn13_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='cf6fce7f-748f-41cf-b347-e324c29bb0c1', metadata={'source': 'tagged_isbn13_description.txt'}, page_content='9780067575208 First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening t

In [10]:
books[books["isbn13"]==int(res[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_with_subtile,tagged_isbn13_description
4581,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [13]:
def retrieve_semantic_recommendations(q,top_k=10) -> pd.DataFrame:

    books_id_list=[]

    res=db_books.similarity_search(q, k=top_k)
    for i in range(0,len(res)) : 
        books_id_list.append(int(res[i].page_content.split()[0].strip()))

    return books[books["isbn13"].isin(books_id_list)]

In [14]:
query = "A book to teach children about nature"
retrieve_semantic_recommendations(query)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_with_subtile,tagged_isbn13_description
497,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,The Sense of Wonder,9780067575208 First published more than three ...
791,9780140448009,0140448004,Three Tales,Gustave Flaubert;Roger Whitehouse;Geoffrey Wall,Fiction,http://books.google.com/books/content?id=XFzga...,Features short fiction by the French naturalis...,2005.0,3.71,110.0,3050.0,Three Tales,9780140448009 Features short fiction by the Fr...
2354,9780394895833,0394895835,Baby's Animal Friends,Phoebe Dunn,Juvenile Fiction,http://books.google.com/books/content?id=ytIAZ...,Photographs capture the special relationship b...,1988.0,3.40,28.0,15.0,Baby's Animal Friends,9780394895833 Photographs capture the special ...
3591,9780670036653,067003665X,The Roald Dahl Treasury,Roald Dahl,Juvenile Fiction,http://books.google.com/books/content?id=NJfZA...,Presents selections from the author's children...,2003.0,4.43,448.0,2091.0,The Roald Dahl Treasury,9780670036653 Presents selections from the aut...
4369,9780761316800,0761316809,It's Winter!,Linda Glaser,Juvenile Nonfiction,http://books.google.com/books/content?id=eSCmK...,A child observes the coming of winter and its ...,2002.0,3.63,32.0,80.0,It's Winter!,9780761316800 A child observes the coming of w...
4581,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
4582,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,"9780786808373 Introducing your baby to birds, ..."
4583,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380 Introduce your babies to birds, ..."
4584,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397 Introduce your baby to birds, ca..."
4645,9780789458209,0789458209,Tree,David Burnie,Juvenile Nonfiction,http://books.google.com/books/content?id=Qwsqj...,Photographs and text explore the anatomy and l...,2000.0,4.07,64.0,5.0,Tree,9780789458209 Photographs and text explore the...
